# Filtrer navn fra bøker

In [1]:
import dhlab.nbtext as nb
import dhlab.token_map as tm
from dhlab.module_update import update, css
import dhlab.graph_networkx_louvain as gnl
import json
import re
import sys
%matplotlib inline
css()

Importer filter fra token_map som ligger i modules. Det vil bli litt patching før ny versjon av `dhlab`.

### Ekstra kommandoer fra `token_map`

* names_from_corpus(korpus)
* count_names_corpus(korpus, token_map)
* names_from_excel(excelfil)
    

In [2]:
update('token_map', overwrite=True)
from token_map import filter_names, names_from_corpus, names_from_excel

Updated file `C:\Users\larsj\Documents\GitHub\NER\token_map.py`

### Gazetter

Gazettene ligger her, følger med repoet fra Github.

In [3]:
first_names = json.load(open("NER-gazetter/first_names.json"))
last_names = json.load(open("NER-gazetter/last_names.json"))
full_names = json.load(open("NER-gazetter/name_structure.json"))
male_names = json.load(open("NER-gazetter/male_names.json"))
female_names = json.load(open("NER-gazetter/female_names.json"))

In [4]:
first_names = list(set(first_names) - {'Oslo'})

### Korpus

Hent noen bøker...

In [5]:
korpus = nb.pure_urn("""
URN

 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015110908267
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009061903027
 ikke digitalisert
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009072800065
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010063006072
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009042204078
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014010708163
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009072800062
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006111501039
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006111501043
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006112001058
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006111701045
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006111601041
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006113000001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014121608238 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008070804046
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017061407030
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013073108136
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009051210001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009051212001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009051213001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041703007
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009020503016
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009032003002
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012041808082
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012100907130
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008040204004
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006082200053 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008073100001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008072304027
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008070201046
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015111007004
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008040804176
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014082708025
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014102108092
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015110507024
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014082508209
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008080500028
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017031348020
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010053106106
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017081548056
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006120500036
 https://urn.nb.no/URN:NBN:no-nb_digibok_2018022807099
 https://urn.nb.no/URN:NBN:no-nb_digibok_2006121301027
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017072007064
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009042204098
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016070748069
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012050708083
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016041908011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016041548100
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008072304028
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008073100001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009062204014
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016040608162
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014080508168
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007011801005
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007011801004
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007021501117
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007010300018
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007010300045
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007010500022
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007011201040
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014082108108
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009062201011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013110508162
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014080508169
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041404093
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009061701068
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010061806078
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013110508163
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008042300025
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008042800109
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041704061
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009042200011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007060504048
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007042301017
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007030201038
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008051904054
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007031501098
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007032801122
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007021901144
https://urn.nb.no/URN:NBN:no-nb_digibok_2008051404001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012021608266 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016103148510 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041704038
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009060204081
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009052904033
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008051904146
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015012008072 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009061504071 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008051904126
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013101708014
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013101708013
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007032101125
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041704058 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009062204039
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007042401029
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014070306049
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016092948064
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041404091
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007052504014
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009042200012 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017062648189
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016101048159
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009061701063
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011020106014
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007072300005
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011120808007
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012030208136 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007071700077
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012053108037
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011111608068
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014021807021
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013102407331
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012022008023
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012022208159
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007072601080
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007070500096
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007070201034
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008052701029
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007060501039
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007060700028
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007060701007
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008111904058
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013012205070
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013030406047
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013040907068
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012100907038
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013102407066
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007062200011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014030308499
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007061804087
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013062706017
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012091805011 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012071905088
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008060304065
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007070304070
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007070900076
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008060300007
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008052304109
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013061706091
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014071508124
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011121405002
 https://urn.nb.no/URN:NBN:no-nb_digibok_2016101048269
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014082708002
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017052207340
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017052207338
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017073107131
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014031108214
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012062505011
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015072308349 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012012708020
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008070400042 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013102407067 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013070205080
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012070205032
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012070205030
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007091901004  
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013093006082 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015060808049
 https://urn.nb.no/URN:NBN:no-nb_digibok_2007112600058
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012101908136
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011020308070
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012101908137
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012101908135
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010121708024
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013011607286
 https://urn.nb.no/URN:NBN:no-nb_digibok_2017022748048
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008010300041 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008030704064
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008031304026
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008030700046
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008031304021
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008081204044
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008020600003
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015041008041
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010100508090
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008031101012 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014061608252
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010060203030 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010072703001
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009102700026
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008080804146
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009011900032
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008092300090
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009012204024
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010011100089
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009041404041
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009030204015
 https://urn.nb.no/URN:NBN:no-nb_digibok_2010122120006 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008120800099 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2008112000056
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009012904103
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009012200018
 https://urn.nb.no/URN:NBN:no-nb_digibok_2009062601017
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011083106080
 https://urn.nb.no/URN:NBN:no-nb_digibok_2011070505043
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014120907542
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013071705100
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014111007502
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012062505101
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013042408089 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2013011408109
 https://urn.nb.no/URN:NBN:no-nb_digibok_2015111608028 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2012102907161
 https://urn.nb.no/URN:NBN:no-nb_digibok_2014022706007 
 https://urn.nb.no/URN:NBN:no-nb_digibok_2018110677040
ikke digitalisert
""")

## Tester med et lite korpus

In [6]:
subkorp = korpus[40:45]
subkorp

['2006120500036',
 '2018022807099',
 '2006121301027',
 '2017072007064',
 '2009042204098']

Generer navn med nb.names()

In [ ]:
korpusnavn = names_from_corpus(subkorp)

Se på names med tm.show_names().

In [ ]:
tm.show_names(korpusnavn)

Filtrer og rensk opp navnelisten med `filter_names()` importert fra `token_map`. Variablene `first_names` og `last_names` slås sammen til en liste som en felles gazett.

In [ ]:
filtered_names = filter_names(korpusnavn, first_names + last_names)


In [ ]:
#her ser vi navnene som er fjernet
tm.show_names(filtered_names['removed'])

In [ ]:
#her ser vi navnene som er beholdt
filtered_names['filtered']

`filtered_names` er en dict med to attributt, ett `filtered` og ett `removed`, sånn at man kan se hva som er fjernet og hvordan det filtrerte funkerer. Om det er fjernet for mye kan det bare legges til i gazetten og hele filtreringen kan kjøres på nytt. Merk at 4-ordsnavnene er bedre, men det mangler noe fremdeles på formen. Litt navnegrammatikk kunne gjøre susen her. Det er greit å få til. Nedenfor ser vi at det blir feil med 4-ordsnavn på formen "fornavn fornavn etternavn fornavn" som kan lukes vekk med en grammatikk.

Lagre de filtrerte navnene til en excelfil.

In [ ]:
nb.frame(nb.frame(filtered_names['filtered']).transpose().sum(axis=1)).to_excel('navn_filter.xls')

Hent de redigerte navnene tilbake. Det kan legges til navn og slettes navn fra filen. 
Endre filnavn i tilfelle lagring fra Excel har endret på det.

In [ ]:
neo_names = navn_excel('navn_filter.xls')


Det tar litt tid å beregne `token_map` så greit å legge den i variabel

In [ ]:
neo_token_map = tm.token_map(neo_names)


### Data fra boken

Tell opp navn med `tm.count_name_strings` ... det ser ut som det fremdeles trengs til litt opprydding. Spesielt de lange navne lager litt ballade, men det er mulig å fjerne dem helt. Nettverk under der igjen. Bøker som ikke er med i grunndataene vil generere en melding om feil.

In [ ]:
opptelling = tell_navn(subkorp, neo_token_map)

In [ ]:
opptelling.head()